In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from pathlib import Path

In [64]:
le_data_df = pd.read_csv("/kaggle/input/encoded-train-dataset/le_dima.csv", index_col=0)
le_data_df = le_data_df.drop(columns=["registration_init_time", "expiration_date"])

In [65]:
data_df = pd.read_csv("/kaggle/input/unencoded-data/dima.csv").iloc[:, 1:]
data_df = data_df.drop(columns=["registration_init_time", "expiration_date"])

In [66]:
raw_song_id = data_df['song_id'].values

In [67]:
song_features = pd.read_csv("/kaggle/input/unencoded-data/extra_song_features.csv", index_col=0)
act_song_features = song_features[song_features["song_id"].isin(raw_song_id)]

In [69]:
from sklearn.preprocessing import LabelEncoder
msno_le = LabelEncoder()
song_id_le = LabelEncoder()

msno_le.fit(data_df['msno'])
song_id_le.fit(data_df['song_id'])

LabelEncoder()

In [ ]:
data_df['msno'] = msno_le.transform(data_df['msno'])
data_df['song_id'] = song_id_le.transform(data_df['song_id'])
act_song_features['song_id'] = song_id_le.transform(act_song_features['song_id'])

In [129]:
merged_data = data_df.merge(act_song_features, on='song_id')

In [130]:
dr_merged_data = merged_data.drop(columns=['genre_ids'])

In [136]:
obj_cols = dr_merged_data.select_dtypes(include="object").columns
label_encoders = {}
for col in obj_cols:
    le = LabelEncoder()
    le.fit(dr_merged_data[col])
    label_encoders[col] = le
    dr_merged_data[col] = le.transform(dr_merged_data[col])

In [137]:
X = dr_merged_data.drop(columns=['target'])
y = dr_merged_data['target']

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
from catboost import CatBoostClassifier
cls = CatBoostClassifier()
cls.fit(X_train, y_train)

In [140]:
from sklearn.metrics import accuracy_score
y_pred = cls.predict(X_test)
accuracy_score(y_test, y_pred)

0.65864

In [162]:
user_info = dr_merged_data[merged_data['msno'] == 230].iloc[0, 6:-10].T.values
song_info = dr_merged_data[merged_data['msno'] == 230].iloc[0, -10:].T.values
obj = np.array([230, 832, -1, -1, -1]+list(user_info)+list(song_info))

In [ ]:
from sklearn.metrics import confusion_matrix

def metric(user_id, predicted, score = "accuracy"):
    y_true = []
    for song_id id in predicted['song_id']:
        if song_id in merged_data[merged_data["msno"] == user_id]['song_id'].values:
            y_true.append(merged_data[merged_data["msno"] == user_id][merged_data["song_id"] == song_id]['target'].values)
        else:
            user_info = merged_data[merged_data['msno'] == user_id].iloc[0, 6:-10].T.values
            song_info = merged_data[merged_data['msno'] == song_id].iloc[0, -10:].T.values
            obj = np.array([user_id, song_id, "new", "new", "new"]+user_info+song_info)
            y_true.append(cls.predict(obj))
    tn, fp, fn, tp = confusion_matrix(y_true, predicted['predict'].values).ravel()

    if score == "accuracy": return round((tp+tn)/(tp+tn+fp+fn), 2)
    elif score == "presicion": return round(tp/(tp+fp), 2)
    elif score == "recall": return round(tp/(tp+tn), 2)